### Import

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)
import warnings ; warnings.filterwarnings('ignore')

### 상권정보
#### $\blacktriangleright$ 서울 자치구별 데이터 처리
동일년도 각 부문별 발생한 평균 매출금액을 의미한다.
- [추정매출] http://data.seoul.go.kr/dataList/OA-15572/S/1/datasetView.do
- [시군구 맵핑 데이터] https://golmok.seoul.go.kr/introduce.do

In [2]:
store = pd.DataFrame()

for i in range(2017,2021):
    ease = pd.read_csv(f'RawData/서울시우리마을가게_상권분석/서울시_우리마을가게_상권분석서비스(신_상권_추정매출)_{i}년.csv', encoding='cp949')
    store = pd.concat([store, ease])
    
store.reset_index(drop=True, inplace=True)

In [3]:
#상권코드와 시군구_행정동 merge를 위한 데이터는 pdf형식에서 csv 파일로 변환해 사용한다.
seoul_market = pd.read_csv('RawData/서울시우리마을가게_상권분석/seoul_v2.csv',encoding='UTF-8')
seoul_market = seoul_market.dropna(axis=0)
seoul_market['우리마을가게 상권분석 서비스 상권영역'] = seoul_market['우리마을가게 상권분석 서비스 상권영역'].apply(lambda x : x.split()[1:4])
seoul_market['자치구'] = seoul_market['우리마을가게 상권분석 서비스 상권영역'].apply(lambda x : str(x[:1]).replace("[","").replace("]","").replace("'",""))
seoul_market['행정동'] = seoul_market['우리마을가게 상권분석 서비스 상권영역'].apply(lambda x : str(x[1:2]).replace("[","").replace("]","").replace("'",""))
seoul_market['상권_코드'] = seoul_market['우리마을가게 상권분석 서비스 상권영역'].apply(lambda x : str(x[2:3]).replace("[","").replace("]","").replace("'",""))
seoul_market = seoul_market.iloc[1:,1:]
seoul_market = seoul_market.reset_index(drop=True)

seoul = ['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구',
         '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구',
         '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구']
seoul_market = seoul_market[seoul_market['자치구'].apply(lambda x : x in seoul) == True]
seoul_market['상권_코드'] = seoul_market['상권_코드'].astype(int)

In [4]:
#상권데이터와 시군구 행정동 데이터를 병합한다.
store = pd.merge(store, seoul_market, how = 'left', on = '상권_코드').rename(columns={'기준_년_코드':'년도'})
store.drop(['상권_구분_코드','상권_구분_코드_명','상권_코드','서비스_업종_코드'], axis=1, inplace=True)

In [5]:
food = ['한식음식점', '치킨전문점', '분식전문점', '호프-간이주점', '커피-음료', '편의점', '육류판매', '제과점', 
        '패스트푸드점','일식음식점', '미곡판매', '슈퍼마켓', '양식음식점','반찬가게', '중식음식점', '수산물판매'] # '전자상거래업'
store = store.query('서비스_업종_코드_명 in @food')

In [6]:
store['분기당_매출금액_평균'] = store['분기당_매출_금액']/store['분기당_매출_건수']
store['주중_매출금액_평균'] = store['주중_매출_금액']/store['주중_매출_건수']
store['주말_매출금액_평균'] = store['주말_매출_금액']/store['주말_매출_건수']
store['월요일_매출금액_평균'] = store['월요일_매출_금액']/store['월요일_매출_건수']
store['화요일_매출금액_평균'] = store['화요일_매출_금액']/store['화요일_매출_건수']
store['수요일_매출금액_평균'] = store['수요일_매출_금액']/store['수요일_매출_건수']
store['목요일_매출금액_평균'] = store['목요일_매출_금액']/store['목요일_매출_건수']
store['금요일_매출금액_평균'] = store['금요일_매출_금액']/store['금요일_매출_건수']
store['토요일_매출금액_평균'] = store['토요일_매출_금액']/store['토요일_매출_건수']
store['일요일_매출금액_평균'] = store['일요일_매출_금액']/store['일요일_매출_건수']
store['시간대_00~06_매출금액_평균'] = store['시간대_00~06_매출_금액']/store['시간대_건수~06_매출_건수']
store['시간대_06~11_매출금액_평균'] = store['시간대_06~11_매출_금액']/store['시간대_건수~11_매출_건수']
store['시간대_11~14_매출금액_평균'] = store['시간대_11~14_매출_금액']/store['시간대_건수~14_매출_건수']
store['시간대_14~17_매출금액_평균'] = store['시간대_14~17_매출_금액']/store['시간대_건수~17_매출_건수']
store['시간대_17~21_매출금액_평균'] = store['시간대_17~21_매출_금액']/store['시간대_건수~21_매출_건수']
store['시간대_21~24_매출금액_평균'] = store['시간대_21~24_매출_금액']/store['시간대_건수~24_매출_건수']
store['남성_매출금액_평균'] = store['남성_매출_금액']/store['남성_매출_건수']
store['여성_매출금액_평균'] = store['여성_매출_금액']/store['여성_매출_건수']
store['연령대_10_매출금액_평균'] = store['연령대_10_매출_금액']/store['연령대_10_매출_건수']
store['연령대_20_매출금액_평균'] = store['연령대_20_매출_금액']/store['연령대_20_매출_건수']
store['연령대_30_매출금액_평균'] = store['연령대_30_매출_금액']/store['연령대_30_매출_건수']
store['연령대_40_매출금액_평균'] = store['연령대_40_매출_금액']/store['연령대_40_매출_건수']
store['연령대_50_매출금액_평균'] = store['연령대_50_매출_금액']/store['연령대_50_매출_건수']
store['연령대_60_이상_매출금액_평균'] = store['연령대_60_이상_매출_금액']/store['연령대_60_이상_매출_건수']

In [7]:
store_sales = store.groupby(['년도','자치구'])[['점포수']+[i for i in store.columns if '평균' in i]].sum()
store_sales = store_sales.fillna(0)
store_sales.columns = ['점포수','연도별_평균_상권_매출금액','연도별_주중_평균_상권_매출금액','연도별_주말_평균_상권_매출금액',
                        '연도별_월요일_평균_상권_매출금액','연도별_화요일_평균_상권_매출금액','연도별_수요일_평균_상권_매출금액',
                        '연도별_목요일_평균_상권_매출금액','연도별_금요일_평균_상권_매출금액','연도별_토요일_평균_상권_매출금액',
                        '연도별_일요일_평균_상권_매출금액','연도별_00_06_시간대_평균_상권_매출금액','연도별_06_11_시간대_평균_상권_매출금액',
                        '연도별_11_14_시간대_평균_상권_매출금액','연도별_14_17_시간대_평균_상권_매출금액',
                        '연도별_17_21_시간대_평균_상권_매출금액','연도별_21_24_시간대_평균_상권_매출금액',
                        '연도별_남성_평균_상권_매출금액','연도별_여성_평균_상권_매출금액','연도별_10대_평균_상권_매출금액',
                        '연도별_20대_평균_상권_매출금액','연도별_30대_평균_상권_매출금액','연도별_40대_평균_상권_매출금액',
                        '연도별_50대_평균_상권_매출금액','연도별_60대이상_평균_상권_매출금액']
store_sales = store_sales.reset_index()
print(store_sales.shape)

(100, 27)


#### $\blacktriangleright$ 광진구 행정동별 데이터 처리
- 서울 자치구별 데이터를 재사용한다.

In [8]:
GJ_dong = ['화양동', '군자동', '중곡1동', '중곡2동', '중곡3동', '중곡4동', '능동', '광장동', '자양1동',
           '자양2동', '자양3동', '자양4동', '구의1동', '구의2동', '구의3동']
GJ_store = store.query('년도==2020 & 행정동 in @GJ_dong')

In [9]:
GJ_store_sales = GJ_store.groupby('행정동')[['점포수']+[i for i in store.columns if '평균' in i]].mean()
GJ_store_sales = GJ_store_sales.fillna(0)
GJ_store_sales.columns = store_sales.columns[2:]
GJ_store_sales = GJ_store_sales.reset_index()
print(GJ_store_sales.shape)

(15, 26)


## Save Data

In [14]:
# 각 데이터를 저장한다.
store_sales.to_csv('ProcessedData/경제정보_seoul_food.csv', index=False)
GJ_store_sales.to_csv('ProcessedData/경제정보_gwangjin_food.csv', index=False)